In [1]:
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from san_18_04_2 import Recognition_Module

/data2/home/shishirm/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
device = 'cuda:0'
torch.cuda.empty_cache()

In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_path = self.data_frame.iloc[idx, 0]
        label = [x for x in self.data_frame.iloc[idx, 5:]] # license plate chracters' indices

        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

In [4]:
transform = transforms.Compose([
    transforms.Resize((480, 480)),
    transforms.ToTensor(),
])

In [5]:
def evaluate(model, dataset_path):
    dataset = CustomDataset(dataset_path, transform=transform)
    dataset_loader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=25, prefetch_factor=10)
    ch_correct = 0
    li_correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        with tqdm(total = len(dataset_loader.dataset)) as pbar:
            for image, label in dataset_loader:
                image = image.to(device)
                label = label.to(device)

                attn, pred_label = model(image)

                predictions = [torch.argmax(curr, dim=1) for curr in pred_label]
                pred_license = torch.stack(predictions, dim=1).to(device)
                ch_equal = (label == pred_license)
                ch_correct += torch.sum(ch_equal).item()
                li_equal = torch.all(ch_equal, dim=1)
                li_correct += torch.sum(li_equal).item()
                total += image.shape[0]

                pbar.update(image.shape[0])
                
    print(f"Character Accuracy : {100 * ch_correct / (7 * total)} \tLicense Accuracy : {100 * li_correct / total}")

In [6]:
model = Recognition_Module()
model.to(device)
model.load_state_dict(torch.load('./model/prediction_module_30.pth', map_location=device))

<All keys matched successfully>

In [7]:
print("Results for CCPD Blur :")
evaluate(model, '../datasets/ccpd_blur.csv')

# e50 Character Accuracy : 76.78216209097778 	License Accuracy : 29.387220416282567
# e30 Character Accuracy : 76.78632075798637 	License Accuracy : 29.382368638105866

Results for CCPD Blur :


  0%|          | 0/20611 [00:00<?, ?it/s]

Character Accuracy : 76.78632075798637 	License Accuracy : 29.382368638105866


In [8]:
print("Results for CCPD Challenge :")
evaluate(model, '../datasets/ccpd_challenge.csv')

# e50 Character Accuracy : 80.11890715128521 	License Accuracy : 35.04389736615803
# e30 Character Accuracy : 80.11176472268806 	License Accuracy : 34.991900485970845


Results for CCPD Challenge :


  0%|          | 0/50003 [00:00<?, ?it/s]

Character Accuracy : 80.11176472268806 	License Accuracy : 34.991900485970845


In [9]:
print("Results for CCPD DB :")
evaluate(model, '../datasets/ccpd_db.csv')

# e50 Character Accuracy : 80.9895098979189 	License Accuracy : 36.81405448085275
# e30 Character Accuracy : 80.93734137950482 	License Accuracy : 36.70548756415318


Results for CCPD DB :


  0%|          | 0/10132 [00:00<?, ?it/s]

Character Accuracy : 80.93734137950482 	License Accuracy : 36.70548756415318


In [10]:
print("Results for CCPD Rotate :")
evaluate(model, '../datasets/ccpd_rotate.csv')

# e50 Character Accuracy : 92.85785337710136 	License Accuracy : 69.01422460956928
# e30 Character Accuracy : 92.87916897585653 	License Accuracy : 69.06396100666468


Results for CCPD Rotate :


  0%|          | 0/10053 [00:00<?, ?it/s]

Character Accuracy : 92.87916897585653 	License Accuracy : 69.06396100666468


In [11]:
print("Results for CCPD Tilt :")
evaluate(model, '../datasets/ccpd_tilt.csv')

# e50 Character Accuracy : 87.38322175573963 	License Accuracy : 50.96968493513371
# e30 Character Accuracy : 87.36951094973335 	License Accuracy : 50.90349483717236


Results for CCPD Tilt :


  0%|          | 0/30216 [00:00<?, ?it/s]

Character Accuracy : 87.36951094973335 	License Accuracy : 50.90349483717236


In [12]:
print("Results for CCPD Weather :")
evaluate(model, '../datasets/ccpd_weather.csv')

# e50 Character Accuracy : 98.48842027059848 	License Accuracy : 91.91919191919192
# e30 Character Accuracy : 98.47841927049848 	License Accuracy : 91.87918791879189


Results for CCPD Weather :


  0%|          | 0/9999 [00:00<?, ?it/s]

Character Accuracy : 98.47841927049848 	License Accuracy : 91.87918791879189


In [13]:
print("Results for CCPD Base Test :")
evaluate(model, '../datasets/test.csv')

# e50 Character Accuracy : 99.12903310293267 	License Accuracy : 95.09491824863748
# e30 Character Accuracy : 99.12308062277228 	License Accuracy : 95.06158435973933


Results for CCPD Base Test :


  0%|          | 0/59999 [00:00<?, ?it/s]

Character Accuracy : 99.12308062277228 	License Accuracy : 95.06158435973933


In [14]:
print("Results for CCPD FN :")
evaluate(model, '../datasets/ccpd_fn.csv')

# e50 Character Accuracy : 75.27815819416907 	License Accuracy : 31.19664234272905
# e30 Character Accuracy : 75.26384999557128 	License Accuracy : 31.11556255067487


Results for CCPD FN :


  0%|          | 0/20967 [00:00<?, ?it/s]

Character Accuracy : 75.26384999557128 	License Accuracy : 31.11556255067487
